In [3]:
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop

Using TensorFlow backend.


In [4]:
#import pandas as pd
#x = pd.read_csv("len40_mask3.txt", header=None, delim_whitespace=True, dtype=str, na_filter=False)

In [5]:
text_length = 40
vocab_size = None

In [6]:
input_shape = (text_length, vocab_size)

In [7]:
#x.head()

In [8]:
#x.values

In [9]:
#from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OneHotEncoder

In [10]:
#le = LabelEncoder()
#categorical = le.fit_transform(x.values.flatten())

In [11]:
#[i for i in x.values.flatten() if not isinstance(i, str)]

In [12]:
from keras.preprocessing.text import Tokenizer

In [13]:
#t = Tokenizer()


In [14]:
#t.fit_on_sequences(x.values)

In [15]:
#t.sequences_to_matrix([["Hello", "this", "is", "test"]])

In [16]:
#t.word_index = None

In [17]:
t = Tokenizer(filters='\n', num_words=vocab_size) #try setting vocab_size to 2000 or 4000 if training not working
with open("len40.txt") as f:
    t.fit_on_texts([f.read(), '<m>'])

In [18]:
X = []
with open("len40_mask3.txt") as f:
    for line in f:
        X.append(t.texts_to_sequences([line])[0])

In [19]:
X = np.array(X) #turn our data from python list of lists into np array

In [20]:
X_filled = []
with open("len40.txt") as f:
    for line in f:
        X_filled.append(t.texts_to_sequences([line])[0])


In [21]:
X_filled = np.array(X_filled)

In [22]:
#from collections import Counter
#Counter([len(p) for p in X])

In [23]:
from sklearn.preprocessing import OneHotEncoder
OHE = OneHotEncoder()
OHE.fit(np.array(list(t.word_index.values())).reshape((-1,1)))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [24]:
OHE.transform(X_filled[0].reshape((-1,1))).todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
reverse_word_map = dict(map(reversed, t.word_index.items())) #turn our numbers into words

In [26]:
[reverse_word_map[i] for i in X[0]] #test

TypeError: 'numpy.int32' object is not iterable

In [27]:
from keras.layers import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed

In [28]:
if vocab_size is None:
    vocab_size = len(t.word_index)

In [29]:
#g = Sequential()

In [30]:
#g.add(Embedding(vocab_size, 300, input_length=text_length))

In [31]:
#g.add(Bidirectional(LSTM(300, return_sequences=True)))

In [32]:
#g.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

In [33]:
#g.summary()

In [34]:
#d = Sequential()

In [35]:
#d.add(Embedding(vocab_size, 300, input_length=text_length))

In [36]:
#d.summary()

In [37]:
from keras.layers import Input, Embedding, LSTM, Dense, Concatenate
from keras.models import Model

In [38]:
main_input_g = Input(shape=(text_length,vocab_size))

In [39]:
#embedding_layer = Embedding(vocab_size, 300, input_length=text_length)
embedding_layer = TimeDistributed(Dense(300))

In [40]:
x = embedding_layer(main_input_g)

In [41]:
x = Bidirectional(LSTM(300, return_sequences=True))(x)

In [42]:
main_output_g = TimeDistributed(Dense(vocab_size, activation='softmax'))(x)

In [43]:
generator_model = Model(inputs=[main_input_g], outputs=[main_output_g])

In [44]:
generator_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 97491)         0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 40, 300)           29247600  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40, 600)           1442400   
_________________________________________________________________
time_distributed_2 (TimeDist (None, 40, 97491)         58592091  
Total params: 89,282,091
Trainable params: 89,282,091
Non-trainable params: 0
_________________________________________________________________


In [71]:
main_input_d = Input(shape=(text_length,vocab_size))
aux_input_d = Input(shape=(text_length,vocab_size))

In [72]:
filled_in = embedding_layer(main_input_d)
context = embedding_layer(aux_input_d)

In [73]:
combined = Concatenate(axis=-1)([filled_in, context])

In [74]:
x = Bidirectional(LSTM(300, return_sequences=True))(combined)

In [75]:
main_output_d = TimeDistributed(Dense(1, activation='sigmoid'))(x) #or sigmoid

In [76]:
discriminator_model = Model(inputs = [main_input_d, aux_input_d], outputs=[main_output_d])

In [77]:
discriminator_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 40, 97491)    0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 40, 97491)    0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 40, 300)      29247600    input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 40, 600)      0           time_distributed_1[3][0]         
          

In [90]:
#AM = Sequential()
optimizer = RMSprop(lr=0.0001, decay=3e-8)
AM_input = Input(shape=(text_length,vocab_size))
generator_output = generator_model(AM_input)
discriminator_output = discriminator_model([generator_output, AM_input])
AM = Model(inputs=[AM_input], outputs=discriminator_output)
#AM.add(discriminator_model)
AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
    metrics=['accuracy'])
AM.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 40, 97491)    0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 40, 97491)    89282091    input_8[0][0]                    
__________________________________________________________________________________________________
model_4 (Model)                 (None, 40, 1)        31410601    model_1[3][0]                    
                                                                 input_8[0][0]                    
Total params: 91,445,092
Trainable params: 91,445,092
Non-trainable params: 0
__________________________________________________________________________________________________


In [87]:
import gc
def train(train_steps=2000, batch_size=10, save_interval=0):
    def categorical_to_one_hot(categorical):
        return OHE.transform(categorical.reshape(batch_size*text_length,1)).toarray().reshape(batch_size,text_length,-1)
    noise_input = None
    if save_interval>0:
        noise_input = np.random.normal(0, 1, size=[16, 100])
    for i in range(train_steps):
        if i%2 == 0:
            gc.collect()
            print(i)
        indices_fake = np.random.randint(0, X.shape[0], size=batch_size)
        fake_unfilled = X[indices_fake, :]
        #noise = np.random.normal(0, 1, size=[batch_size, 100])
        fake_unfilled = categorical_to_one_hot(fake_unfilled)
        fake_filled = generator_model.predict(fake_unfilled)

        indices_real = np.random.randint(0, X.shape[0], size=batch_size)
        real_unfilled = X[indices_real, :]
        real_filled = X_filled[indices_real, :]
        real_unfilled = categorical_to_one_hot(real_unfilled)
        real_filled = categorical_to_one_hot(real_filled)
        
        x_unfilled = np.concatenate((real_unfilled, fake_unfilled))
        x_filled = np.concatenate((real_filled, fake_filled))
        y = np.ones([2*batch_size, text_length, 1])
        y[batch_size:, :, :] = 0
        d_loss = discriminator_model.train_on_batch([x_unfilled, x_filled], y)
        #above here is training the discriminator (DM)
        #under here is training the generator (AM)
        y = np.ones([batch_size, text_length, 1])
        #noise = np.random.normal(0, 1, size=[batch_size, 100])
        indices_fake = np.random.randint(0, X.shape[0], size=batch_size)
        fake_unfilled = X[indices_fake, :]
        fake_unfilled = categorical_to_one_hot(fake_unfilled)
        a_loss = AM.train_on_batch(fake_unfilled, y)
        log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
        log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
        print(log_mesg)
        #if save_interval>0:
        #    if (i+1)%save_interval==0:
        #        self.plot_images(save2file=True, samples=noise_input.shape[0],\
        #            noise=noise_input, step=(i+1))

In [91]:
optimizer = RMSprop(lr=0.0002, decay=6e-8)

discriminator_model.compile(loss='binary_crossentropy', optimizer=optimizer,\
    metrics=['accuracy'])

train()

0
0: [D loss: 0.003498, acc: 1.000000]  [A loss: 0.206233, acc: 0.955000]
1: [D loss: 0.001265, acc: 1.000000]  [A loss: 0.000604, acc: 1.000000]
2
2: [D loss: 0.000938, acc: 1.000000]  [A loss: 0.000570, acc: 1.000000]
3: [D loss: 0.000788, acc: 1.000000]  [A loss: 0.000538, acc: 1.000000]
4
4: [D loss: 0.000689, acc: 1.000000]  [A loss: 0.000504, acc: 1.000000]
5: [D loss: 0.000607, acc: 1.000000]  [A loss: 0.000472, acc: 1.000000]
6
6: [D loss: 0.000566, acc: 1.000000]  [A loss: 0.000478, acc: 1.000000]
7: [D loss: 0.000524, acc: 1.000000]  [A loss: 0.000458, acc: 1.000000]
8
8: [D loss: 0.000504, acc: 1.000000]  [A loss: 0.000409, acc: 1.000000]
9: [D loss: 0.000450, acc: 1.000000]  [A loss: 0.000371, acc: 1.000000]
10
10: [D loss: 0.000421, acc: 1.000000]  [A loss: 0.000364, acc: 1.000000]
11: [D loss: 0.000386, acc: 1.000000]  [A loss: 0.000326, acc: 1.000000]
12
12: [D loss: 0.000383, acc: 1.000000]  [A loss: 0.000315, acc: 1.000000]
13: [D loss: 0.000369, acc: 1.000000]  [A los

KeyboardInterrupt: 

In [68]:
print("test")

test


In [54]:
indices_fake = np.random.randint(0, X.shape[0], size=10)
fake_unfilled = X[indices_fake, :]

In [55]:
fake_unfilled.shape

(10, 40)

In [59]:
one_hot_encoded_test = OHE.transform(fake_unfilled.reshape(400,1)).toarray().reshape(10,40,-1)

[reverse_word_map.get(OHE.active_features_[np.argmax(i)], '<m>') for i in one_hot_encoded_test[0]]

['ran',
 'eos',
 'just',
 'to',
 'find',
 'the',
 'road',
 'had',
 'ended',
 'eos',
 'just',
 'where',
 'it',
 'began',
 'eos',
 'the',
 'next',
 'man',
 '<m>',
 'i',
 'marry',
 'eos',
 'there',
 'ai',
 "n't",
 'nothin',
 "'",
 'he',
 'wo',
 "n't",
 'do',
 'eos',
 'most',
 'likely',
 '<m>',
 'me',
 '<m>',
 'much',
 'eos',
 'the']